In [ ]:
ignore_warnings = True
if ignore_warnings:
    import warnings
    warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np

political_spectrum_dict = {
    'junge Welt':'left',
    'NachDenkSeiten':'left',
    'taz':'left leaning',
    'Süddeutsche Zeitung':'left leaning',
    'stern TV':'left leaning',
    'DER SPIEGEL':'left leaning',
    'Der Tagesspiegel':'center',
    'ARD':'center',
    'ZDF':'center',
    'ZDFheute Nachrichten':'center',
    'Bayerischer Rundfunk':'center',
    'ntv Nachrichten':'center',
    'RTL':'right leaning',
    'FOCUS Online':'right leaning',
    'ZEIT ONLINE':'left leaning',
    'faz':'right leaning',
    'WELT':'right leaning',
    'BILD':'right leaning',
    'NZZ Neue Zürcher Zeitung':'right leaning',
    'Junge Freiheit':'right',
    'COMPACTTV':'right',
}

media = [
    'junge Welt',
    "NachDenkSeiten",
    'taz',
    'Süddeutsche Zeitung',
    'stern TV',
    "DER SPIEGEL",
    'Der Tagesspiegel',
    'ARD',
    'Tagesschau',
    'ZDF',
    "ZDFheute Nachrichten",
    'Bayerischer Rundfunk',
    'ntv Nachrichten',
    'RTL',
    'FOCUS Online',
    'ZEIT ONLINE',
    'faz',
    'WELT',
    "BILD",
    'NZZ Neue Zürcher Zeitung',
    "Junge Freiheit",
    'COMPACTTV'
]

def define_print(verbose=True):
    if verbose:
        verboseprint = print
    else:
        verboseprint = lambda *args: None
    return verboseprint

def get_N_matrix(df, verbose=True, drop_subsumed=True, drop_medium_specific=True):
    verboseprint = define_print(verbose=verbose)
    MEDIA = media
    cv = CountVectorizer(max_df=0.9, min_df=10, max_features=10000, ngram_range=(1, 3))
    topic = df.iloc[0]['topic']
    
    verboseprint("restructuring dataframe with " + str(len(df)) + " transcripts...")
    df["preprocessed"] = df["preprocessed"] + " "
    df = df[["medium", "preprocessed", "topic"]]
    df_grouped = df.groupby(["medium", "topic"]).sum()

    df = pd.DataFrame(index=MEDIA, columns=["preprocessed"])
    empty_media = []
    for medium in MEDIA:
        try:
            df.loc[medium] = df_grouped.loc[medium].loc[topic]["preprocessed"]
        except:
            print(
                medium
                + " does not have any videos categorized under category '"
                + topic
                + "'."
            )
            df.drop(index=medium, inplace=True)
            empty_media.append(medium)

    for empty_medium in empty_media:
        MEDIA.remove(empty_medium)

    verboseprint("fitting cv model for topic " + topic + "...")
    N_matrix = cv.fit_transform(df["preprocessed"].values)

    verboseprint("counting n-gram occurences...")
    N_df = pd.DataFrame(
        data=N_matrix.toarray().transpose(),
        columns=df.index,
        index=cv.get_feature_names_out(),
    )

    if drop_medium_specific:
        verboseprint(
            "dropping medium-specific n-grams that occur in one medium at least 90% of the time..."
        )
        N_sum = N_df.sum(axis=1)
        specific_mask = np.full(len(N_df.index), False)
        mask_df = N_df.apply(lambda x: x>0.9*N_sum)
        for medium in MEDIA:
            specific_mask = specific_mask | mask_df[medium].values
        N_df.drop(N_df.index[specific_mask], inplace=True)

    if drop_subsumed:
        N_df = N_df.reset_index().rename(columns={"index": "phrase"})
        N_df["n_gram"] = N_df["phrase"].apply(str.split).apply(len)
        N_df["count"] = N_df[MEDIA].sum(axis=1)

        monograms = N_df[N_df["n_gram"] == 1]
        bigrams = N_df[N_df["n_gram"] == 2]
        trigrams = N_df[N_df["n_gram"] == 3]
        bigram_words = list(
            set(
                [
                    word
                    for bigram_sublist in bigrams["phrase"].apply(str.split).tolist()
                    for word in bigram_sublist
                ]
            )
        )
        trigram_words = list(
            set(
                [
                    word
                    for trigram_sublist in trigrams["phrase"].apply(str.split).tolist()
                    for word in trigram_sublist
                ]
            )
        )

        verboseprint("extracting subsumed n-grams...")
        monograms_in_bigrams = monograms[monograms["phrase"].isin(bigram_words)]
        monograms_in_trigrams = monograms[monograms["phrase"].isin(trigram_words)]

        bigrams_in_trigrams_words = list(
            set(
                [
                    bigram_word
                    for bigram_word in bigram_words
                    if bigram_word in trigram_words
                ]
            )
        )
        bigrams_in_trigrams_mask = bigrams["phrase"].apply(
            lambda bigram: True
            if bigram.split()[0] in bigrams_in_trigrams_words
            or bigram.split()[1] in bigrams_in_trigrams_words
            else False
        )
        bigrams_in_trigrams = bigrams[bigrams_in_trigrams_mask]

        threshold = 0.7
        verboseprint(
            f"filtering n-grams which are subsumed more than {int(100*threshold)}% of the time..."
        )
        monograms_in_bigrams_above_threshold = list(
            set(
                [
                    monogram["phrase"]
                    for _, monogram in monograms_in_bigrams.iterrows()
                    for _, bigram in bigrams.iterrows()
                    if monogram["phrase"] in bigram["phrase"].split()
                    and bigram["count"] > threshold * monogram["count"]
                ]
            )
        )
        monograms_in_trigrams_above_threshold = list(
            set(
                [
                    monogram["phrase"]
                    for _, monogram in monograms_in_trigrams.iterrows()
                    for _, trigram in trigrams.iterrows()
                    if monogram["phrase"] in trigram["phrase"].split()
                    and trigram["count"] > threshold * monogram["count"]
                ]
            )
        )
        bigrams_in_trigrams_above_threshold = list(
            set(
                [
                    bigram["phrase"]
                    for _, bigram in bigrams_in_trigrams.iterrows()
                    for _, trigram in trigrams.iterrows()
                    if (
                        bigram["phrase"] in " ".join(trigram["phrase"].split()[:2])
                        or bigram["phrase"] in " ".join(trigram["phrase"].split()[-2:])
                    )
                    and trigram["count"] > threshold * bigram["count"]
                ]
            )
        )
        n_grams_above_threshold = list(
            set(
                np.append(
                    np.append(
                        monograms_in_bigrams_above_threshold,
                        monograms_in_trigrams_above_threshold,
                    ),
                    bigrams_in_trigrams_above_threshold,
                )
            )
        )

        N_df.drop(
            N_df[N_df["phrase"].isin(n_grams_above_threshold)].index, inplace=True
        )
        N_df.set_index("phrase", inplace=True)
        N_df.drop(columns=["n_gram", "count"], inplace=True)
    return N_df

def filter_N_by_information_score(N_df, n=1000, verbose=True):
    verboseprint = define_print(verbose=verbose)
    verboseprint("filtering " + str(n) + " most discriminative phrases from sample...")
    n_i = len(N_df.index)
    n_j = len(N_df.columns)
    P_ij = N_df / N_df.to_numpy().sum()
    P_i = P_ij.sum(axis=1)
    P_j = P_ij.sum(axis=0)

    I = np.zeros((n_i, n_j))

    for i in range(n_i):
        for j in range(n_j):
            I[i][j] = P_ij.values[i][j] * np.log2(P_ij.values[i][j] / P_i[i] / P_j[j])

    I = pd.DataFrame(I, index=N_df.index, columns=N_df.columns)
    I = I.fillna(0.0)
    I["sum"] = I.sum(axis=1)
    I.sort_values(by="sum", ascending=False, inplace=True)
    return N_df.loc[I.index[:n]]

In [ ]:
df = pd.read_pickle('../data/topics_combined.pkl')
topic_counts = df['topic'].value_counts()
topic_dict = dict(zip(np.arange(-1,89), topic_counts.index.to_list()))

In [ ]:
whitelist = {}
for i in range(-1,89):
    whitelist[i] = []

In [ ]:
whitelist[7] = [
    'polen',
    'ukrainischen',
    'ukrainische',
    'geflüchtete',
    'migranten',
    'kiew',
    'russischen',
    'libyen',
    'bahnhof',
    'asylbewerber',
    'menschen ukraine',
    'zug',
    'helfen',
    'lampedusa',
    'polnischen',
    'ukrainischen grenze',
    'ukrainer',
    'russland',
    'migrations',
    'krieges',
    'serbien',
    'spenden',
    'kindern',
    'verteilung',
    'einwanderung',
    'schiff',
    'polnisch',
    'russische',
    'unterstützung',
    'zuwanderung',
    'küstenwache',
    'hilfsbereitschaft',
    'balkanroute',
    'westen',
    'frauen kinder',
    'mittelmeer',
    'migrationshintergrund',
    'hafen',
    'boot',
    'fliehen',
    'einwanderer',
    'österreich',
    'camp',
    'nachbarländer',
    'mazedonien',
    'belarus',
    'kosovo',
    'ausländer',
    'kriegsflüchtlinge',
    'eritrea',
    'geflohen',
    'illegal',
    'kroatien',
    'polnischen grenze',
    'afrika',
    'flucht',
    'bomben',
    'grenzübergang',
    'ukrainerinnen',
    'libysche',
    'masseneinwanderung',
    'ukrainisch',
    'libyschen',
    'abgeschoben',
    'albanien',
    'polnische',
    'moria',
    'warschau',
    'schlepper',
    'inseln',
    'migrantinnen',
    'unterbringung',
    'krise',
    'bosnien',
    'griechischen',
    'verteilt',
    'fluchtursachen',
    'willkommenskultur',
    'slowakei',
    'überfahrt',
    'illegale',
    'ukrainischer',
    'unterstützen',
    'obergrenze',
    'deutschen',
    'seenot',
    'einwanderungsgesetz',
    'arbeit',
    'schweiz',
    'männer',
    'verteilen',
    'russen',
    'küste',
    'freiwillige',
    'land verlassen',
    'zaun',
    'zivilisten',
    'ehrenamtliche',
    'arbeiter',
    'flüchtlingskrise',
    'afrikaner',
    'illegalen',
    'rumänien',
    'lesbos',
    'marokko',
    'ankunft',
    'balkan',
    'boote',
    'kriegsflüchtlingen',
    'sicherheit',
    'schiffe',
    'bürger',
    'kind',
    'schweizer',
    'verwandten',
    'flüchtling',
    'einwanderungs',
    'flüchten',
    'ukrainerin',
    'aufzunehmen',
    'malta',
    'rassismus',
    'weißrussland',
    'familien',
    'aufnehmen',
    'reduzieren',
    'abwanderung',
    'ukraine krieg',
    'solidarität',
    'syrer',
    'armut',
    'lebensmittel',
    'ursachen',
    'fachkräfte',
    'asylrecht',
    'afghanen',
    'verfahren',
    'schlauchboot',
    'asylverfahren',
    'asylpolitik',
    'menschenrechte',
    'spanien',
    'mütter',
    'integration',
    'frieden',
    'grundgesetz',
    'frontex',
    'abschiebung',
    'kriege',
    'opfer',
    'flüchtlings',
    'duldung',
    'platz',
    'gruppen',
    'ukraine geflohen',
    'ausbeutung',
    'zuflucht',
    'sozialstaat',
    'afghanistan',
    'bundesinnenministerium',
    'gerettet',
    'familiennachzug',
    'ungarn',
    'irak',
    'zurückgeschickt',
    'bulgarien',
    'arbeitsmarkt',
    'griechen',
    'nachbarland',
    'wohnung',
    'hilfsorganisationen',
    'tunesien',
    'einwanderungsland',
    'lagern',
    'herkunft',
    'nigeria',
    'einwanderungspolitik',
    'flüchtlingskonvention',
    'europäische union',
    'seenotrettung',
    'griechischen inseln',
    'unterkunft',
    'asylsuchende',
    'hilfsgüter',
    'vergewaltigt',
    'bürgerkrieg',
    'abschiebungen',
    'kroatischen',
    'europäischen union',
    'zurückkehren',
    'kriminalität',
    'wirtschaftsflüchtlinge',
    'geflüchtet',
    'flüchtlingslagern',
    'flüchtlinge aufgenommen',
    'asylrechts',
    'asylbewerbern',
    'neuankömmlinge',
    'ertrinken',
    'kultur',
    'eu staaten',
    'ausländern',
    'asylantrag',
    'immigranten',
    'flüchtenden',
    'flüchtlingslager',
    'außengrenzen',
    'litauen',
    'zuwanderer',
    'visa',
    'mitgliedstaaten',
    'flüchtende',
    'überqueren',
    'hilfsorganisation',
    'illegaler',
    'flüchtlingsstrom',
    'festland',
    'meer',
    'unterkünfte',
    'einwanderern',
    'europäische länder',
    'grenzpolizei',
    'nato',
    'zurückgebracht',
    'medizinische versorgung',
    'widerstand',
    'flüchtlingsheim',
    'zelte',
    'staatsbürgerschaft',
    'arbeitskräfte',
    'leistungen',
    'bekämpfung',
    'migrationspolitik',
    'rückkehr',
    'verfolgung',
    'schleppern',
    'humanitäre',
    'bezahlen',
    'täter',
    'menschen migrationshintergrund',
    'proteste',
    'insel lesbos',
    'millionen flüchtlinge',
    'asylanträge',
    'syrischen',
    'illegale migration',
    'verwandte',
    'rechtsstaat',
    'container',
    'migrant',
    'slowenien',
    'angriffskrieg',
    'empfangen',
    'schleuser',
    'kontrollieren',
    'illegale einwanderer',
    'griechischen insel',
    'asylsuchenden',
    'rechtsradikalen',
    'grenzschützer',
    'überfüllt',
    'traumatisiert',
    'flüchtlingsunterkunft',
    'verfolgt',
    'demonstrationen',
    'grenzöffnung',
    'grenzregion',
    'flüchtlingscamp',
    'somalia',
]

In [ ]:
whitelist[10] = [
    'windräder',
    'ausbau',
    'solar',
    'energiewende',
    'wetter',
    'ausbau erneuerbaren energien',
    'pflanzen',
    'windrad',
    'klimakatastrophe',
    'klimabewegung',
    'studie',
    'climate',
    'photovoltaik',
    'wasserstoff',
    'erwärmung',
    'wissenschaftler',
    'erderwärmung',
    'kohlendioxid',
    'wind',
    'eeg',
    'elektroauto',
    'ökostrom',
    'klimaneutralität',
    'klimapolitik',
    'tempolimit',
    'klimadebatte',
    'öko',
    'infrastruktur',
    'atmosphäre',
    'heizung',
    'nachhaltigkeit',
    'treibhausgase',
    'kohleausstieg',
    'rwe',
    'stromerzeugung',
    'verkehr',
    'energiepreise',
    'solaranlage',
    'windkraft',
    'fleisch',
    'gerechtigkeit',
    'windpark',
    'temperatur',
    'kernenergie',
    'offshore',
    'windkraftanlagen',
    'industrie',
    'windenergie',
    'energieversorgung',
    'klimaziele',
    'zertifikate',
    'wasserkraft',
    'klimapaket',
    'umweltschutz',
    'emissionshandel',
    'weltklimarat',
    'braunkohle',
    'kernkraft',
    'kraftwerke',
    'klimaneutral',
    'strompreis',
    'solarzellen',
    'photovoltaikanlagen',
    'solaranlagen',
    'technologie',
    'windparks',
    'fußabdruck',
    'meeresspiegel',
    'methan',
    'fracking',
    'erwärmt',
    'kraftwerk',
    'solarenergie',
    'diesel',
    'erneuerbare energien',
    'folgen klimawandels',
    'energiepolitik',
    'pariser klimaabkommen',
    'atomkraft',
    'stromnetz',
    'energieträger',
    'klimaschutzgesetz',
    'klimagipfel',
    'klima thema',
    'klima paket',
    'klimaaktivisten',
    'klimafreundlicher',
    'erdgas',
    'stromversorgung',
    'grad celsius',
    'kampf klimawandel',
    'ausbau windenergie',
    'biomasse',
    'klimaschutzziele',
    'klimafrage',
    'treibhausgasen',
    'naturschutz',
    'strombedarf',
    'atomkraftwerke',
    'klimakonferenz',
    'weltklimakonferenz',
    'windstrom',
    'eeg umlage',
    'klimaziele erreichen',
    'erneuerbarer energien',
    'treibhausgas',
    'erneuerbaren strom',
    'klimaforscher',
    'greta thunberg',
    'erwärmung grad',
    'fossilen energien',
    'menschengemachten klimawandel',
    'windrädern',
    'geothermie',
    'dekarbonisierung',
    'sonnenenergie',
    'strom produzieren',
    'brennstoffe',
    'kraftwerken',
    'grad ziel',
    'erderwärmung grad',
]

In [ ]:
whitelist[3] = [
    'täter',
    'einsatz',
    'urteil',
    'anklage',
    'antifa',
    'nsu',
    'protest',
    'landgericht',
    'verfassungsschutz',
    'tatverdächtigen',
    'strafbar',
    'akten',
    'leiche',
    'staatsanwaltschaft',
    'beamte',
    'kundgebung',
    'kapitol',
    'bundesgerichtshof',
    'ermittlungen',
    'verurteilt',
    'prozess',
    'verletzt',
    'geldstrafe',
    'strafmaß',
    'strafe',
    'proteste',
    'polizistinnen polizisten',
    'einsatzkräfte',
    'ermittler',
    'anwälte',
    'mörder',
    'kinderpornografie',
    'demonstrationen',
    'demo',
    'festgenommen',
    'staatsanwälte',
    'angeklagten',
    'strafrecht',
    'haft',
    'missbrauch',
    'unfall',
    'rassismus',
    'sexualisierter gewalt',
    'strafverfahren',
    'richterin',
    'beschuldigten',
    'verteidigung',
    'zeugen',
    'unschuldsvermutung',
    'verteidiger',
    'tatort',
    'sexualisierte gewalt',
    'motiv',
    'verdächtige',
    'waffen',
    'bundespolizei',
    'tatverdächtige',
    'gegendemonstranten',
    'drogen',
    'messer',
    'aktivisten',
    'tatbestand',
    'justiz',
    'flucht',
    'betroffenen',
    'angaben polizei',
    'straftaten',
    'schüsse',
    'bundesanwaltschaft',
    'kriminell',
    'todesstrafe',
    'gerichtssaal',
    'geschossen',
    'dna',
    'polizeibeamten',
    'kindesmissbrauch',
    'knast',
    'widerstand',
    'erschossen',
    'querdenken',
    'rechtsextremen',
    'mord',
    'waffe',
    'berufung',
    'staatsanwalt',
    'amtsgericht',
    'munition',
    'verletzungen',
    'querdenker',
    'sicherheitsbehörden',
    'berliner polizei',
    'rechtsprechung',
    'festnahme',
    'protesten',
    'rechtsextremismus',
    'migranten',
    'beihilfe',
    'bewaffnet',
    'angeklagt',
    'verdächtigen',
    'oberlandesgericht',
    'bewährung',
    'staatsanwaltschaften',
    'demonstriert',
    'angeklagte',
    'paragraf',
    'block',
    'krimi',
    'morde',
    'schuld',
    'betroffene',
    'geheimdienst',
    'durchsucht',
    'jury',
    'schuldig',
    'rechtsextreme',
    'ausschreitungen',
    'straftat',
    'polizeisprecher',
    'demonstrieren',
    'eskaliert',
    'gutachten',
    'haftstrafe',
    'raub',
    'mandanten',
    'strafrechtlichen',
    'freigesprochen',
    'vorgeworfen',
    'besitz',
    'räumung',
    'sexuelle gewalt',
    'taten',
    'spuren',
    'verstöße',
    'ermittelt',
    'betrüger',
    'hauptverhandlung',
    'auflagen',
    'attentat',
    'haftbefehl',
    'aussage',
    'tot',
    'straftatbestand',
    'sichergestellt',
    'polizistin',
    'polizeibeamte',
    'streifenwagen',
    'dealer',
    'juristisch',
    'freiheitsstrafe',
    'einsatzkräften',
    'klage',
    'unschuldig',
    'leichen',
    'mordes',
    'vergewaltigung',
    'angreifer',
    'polizeigewalt',
    'aufklärung',
    'zugriff',
    'korruption',
    'kinderpornografische',
    'asyl',
    'sexueller gewalt',
    'schuss',
    'körperverletzung',
    'sexuellen missbrauch',
    'pistole',
    'polizeiwache',
    'verletzte',
    'auseinandersetzungen',
    'rechtskräftig',
    'schwer verletzt',
    'gerecht',
    'tätern',
    'mordfall',
    'gerichte',
    'neonazis',
    'juristischen',
    'bundeskriminalamt',
    'protestieren',
    'polizeigewerkschaft',
    'polizisten einsatz',
    'vorwurf',
    'rassistische',
    'verhandlung',
    'eingestellt',
    'polizeilichen',
    'tötung',
    'strafgesetzbuch',
    'rechtsanwalt',
    'hass',
    'lka',
    'spezialeinheiten',
    'polizeiliche',
    'zeuge',
    'gewehr',
    'töten',
    'randalierer',
    'untergrund',
    'sexuellen missbrauchs',
    'kriminalpolizei',
    'verbrecher',
    'missbraucht',
    'getötet',
    'überfall',
    'missstände',
    'verurteilung',
    'angegriffen',
    'schwarze',
    'berufen',
    'beschuldigte',
    'spurensicherung',
    'festnehmen',
    'kriminalität',
    'gestanden',
    'wasserwerfer',
    'mutmaßliche täter',
    'polizei einsatz',
    'ermordet',
    'ermittlungsverfahren',
    'festnahmen',
    'zeugenaussage',
    'anwältin',
    'haftbefehle',
    'gefasst',
    'beweisen',
    'schusswaffe',
    'kontrolle'
]

In [ ]:
color_palette = [
    'rgb(20.0, 20.0, 255.0)',
    'rgb(50.0, 0.0, 170.0)',
    'rgb(120.0, 120.0, 120.0)',
    'rgb(170.0, 0.0, 50.0)',
    'rgb(255.0, 20.0, 20.0)'
]

n_components=3
scale = True
plot = False

def evaluate():
    scaler = StandardScaler()
    model = TruncatedSVD(n_components=n_components)

    if scale:
        N_scaled = scaler.fit_transform(N.values)
        N_df_trunc = model.fit_transform(N_scaled)
    else:
        N_df_trunc = model.fit_transform(N.values)

    export_df = pd.DataFrame(model.components_[1:], index=['X', 'Y'], columns=media)
    export_df.to_pickle(f'../data/mit_method/{topic_dict[topic_index]}.pkl')

    if plot:
        for n in range(n_components):
            fig = px.scatter(
                x=model.components_[n], 
                y=[0.0 for test in model.components_[n]], 
                color=[political_spectrum_dict[medium] for medium in N.columns.to_list()],
                color_discrete_sequence=color_palette,
                hover_data=[N.columns.to_list()], 
                title=f'Topic: \'{topic}\', {n}. Principal Component',
                )
            fig.show()

    n_phrases = 5
    for axis in range(n_components):
        ind_neg = np.argsort(N_df_trunc[:,axis])[:n_phrases]
        ind_pos = np.argsort(N_df_trunc[:,axis])[-n_phrases:]
        list_neg = N.index[ind_neg].to_list()
        list_pos = N.index[ind_pos].to_list()
        list_pos.reverse()
        print(f'principal component: {axis}, most negative phrases: {list_neg}')
        print(f'principal component: {axis}, most positive phrases: {list_pos}')

    M = N.copy(deep=True)
    M['dummy'] = np.round(M['NachDenkSeiten']).astype(int)
    scaler_m = StandardScaler()
    model_m = TruncatedSVD(n_components=n_components)

    M_scaled = scaler_m.fit_transform(M.values)
    M_df_trunc = model_m.fit_transform(M_scaled)

    export_df_m = pd.DataFrame(model_m.components_[1:], index=['X', 'Y'], columns=media + ['dummy'])
    export_df_m.to_pickle(f'../data/mit_method/{topic_dict[topic_index]}_unrobust.pkl')

In [ ]:
for topic_index in [3,7,10]:
    topic = topic_dict[topic_index]
    print('topic: ' + topic)
    N_df = get_N_matrix(df[df['topic'] == topic]) 
    N_df = filter_N_by_information_score(N_df, n=3000)
    N = N_df[N_df.index.isin(whitelist[topic_index])]
    print(f'words considered after filtering: {len(N.index)}')
    evaluate()